### 作業目的: 熟練Pytorch Dataset與DataLoader進行資料讀取

本此作業主要會使用[IMDB](http://ai.stanford.edu/~amaas/data/sentiment/)資料集利用Pytorch的Dataset與DataLoader進行
客製化資料讀取。
下載後的資料有分成train與test，因為這份作業目的在讀取資料，所以我們取用train部分來進行練習。
(請同學先行至IMDB下載資料)

### 載入套件

In [1]:
# Import torch and other required modules
import glob
import torch
import re
import nltk
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_svmlight_file
from nltk.corpus import stopwords

nltk.download('stopwords') #下載stopwords
nltk.download('punkt') #下載word_tokenize需要的corpus
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Guan-Ting
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Guan-Ting
[nltk_data]     Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 探索資料與資料前處理
在train資料中，有分成pos(positive)與neg(negative)，分別為正評價與負評價，此評價即為label。

In [2]:
# 讀取字典，這份字典為review內所有出現的字詞
with open('aclImdb/imdb.vocab', 'r', encoding='utf-8') as f:
    vocab = f.read().split('\n')

# 以nltk stopwords移除贅字，過多的贅字無法提供有用的訊息，也可能影響模型的訓練
print(f"vocab length before removing stopwords: {len(vocab)}")
vocab = [word for word in vocab if word not in stopwords]
print(f"vocab length after removing stopwords: {len(vocab)}")

# 將字典轉換成dictionary
vocab_dic = {}
for i, word in enumerate(vocab):
    vocab_dic[word] = i
# vocab_dic

vocab length before removing stopwords: 89527
vocab length after removing stopwords: 89356


In [3]:
# 將資料打包成(x, y)配對，其中x為review的檔案路徑，y為正評(1)或負評(0)
# 這裡將x以檔案路徑代表的原因是讓同學練習不一次將資料全讀取進來，若電腦記憶體夠大(所有資料檔案沒有很大)
# 可以將資料全一次讀取，可以減少在訓練時I/O時間，增加訓練速度
review_pairs = []
for tag in ['pos', 'neg']:
    for file in glob.glob(f'./aclImdb/train/{tag}/*.txt'):
        review_pairs.append((file, int(tag=='pos')))

print(review_pairs[:2])
print(f"Total reviews: {len(review_pairs)}")

[('./aclImdb/train/pos\\0_9.txt', 1), ('./aclImdb/train/pos\\10000_8.txt', 1)]
Total reviews: 25000


### 建立Dataset與DataLoader讀取資料
這裡我們會需要兩個helper functions，其中一個是讀取資料與清洗資料的函式(load_review)，另外一個是生成詞向量BoW的函式
(generate_bow)

In [4]:
def load_review(review_path):
    
    with open(review_path, 'r', encoding='utf-8') as f:
        review = f.read()
        
    #移除non-alphabet符號、贅字與tokenize
    review = re.sub('\W', ' ', review)
    review = nltk.word_tokenize(review)
    review = [word for word in review if word not in stopwords]
    return review

In [5]:
def generate_bow(review, vocab_dic):
    bag_vector = np.zeros(len(vocab_dic))
    for word in review:
        if vocab_dic.get(word):
            bag_vector[vocab_dic.get(word)] += 1
            
    return bag_vector

In [6]:
class dataset(Dataset):
    '''custom dataset to load reviews and labels
    Parameters
    ----------
    data_pairs: list
        directory of all review-label pairs
    vocab: list
        list of vocabularies
    '''
    def __init__(self, data_dirs, vocab):
        self.data_dirs = data_dirs
        self.vocab = vocab

    def __len__(self):
        return len(self.data_dirs)

    def __getitem__(self, idx):
        review_path, tag = self.data_dirs[idx]
        review = load_review(review_path)
        bag_vector = generate_bow(review, self.vocab)
        return (bag_vector, tag)

In [7]:
# 建立客製化dataset
custom_dst = dataset(review_pairs, vocab_dic)
custom_dst[10]

(array([0., 2., 1., ..., 0., 0., 0.]), 1)

In [8]:
# 建立dataloader
custom_dataloader = DataLoader(custom_dst, batch_size=4, shuffle=True)
next(iter(custom_dataloader))

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.],
         [0., 4., 0.,  ..., 0., 0., 0.]], dtype=torch.float64),
 tensor([1, 1, 1, 0])]